In [401]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from math import sqrt
import scipy
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
#import statsmodels.formula.api as smf
import copy
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
from sklearn.ensemble import RandomForestRegressor
from math import sqrt
from functools import reduce
import numpy as np
#import ee
#import geemap
import geopandas as gpd
import rioxarray as rxr
import rasterstats as rs
from rasterio.plot import show
import rasterio
import cartopy.crs as ccrs
import gdal
%matplotlib inline
plt.rcParams["font.family"] = "Arial"

In [402]:
ndvi_irri = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/ndvi_irrigated.csv')[['ADM3_EN','yr1','yr5']]
ndvi_rf =pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/ndvi_rainfed.csv')[['ADM3_EN','yr1','yr5']]
ndvi_all = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/ndvi_all.csv')[['ADM3_EN','yr1','yr5']]
#ndvi_all

# 5 YEAR INCREMENTS
## Irrigated Rice


In [403]:
points = '/Users/delatorredm/Library/CloudStorage/OneDrive-TheUniversityofAuckland/Ch4_Climate_Impacts/geospatial_layers/out5yr/intersect_DS_5yr.shp'
#points = '/Users/delatorredm/Library/CloudStorage/OneDrive-TheUniversityofAuckland/Ch4_Climate_Impacts/geospatial_layers/out2017/intersect_DS_2017.shp'
points = gpd.read_file(points)

startYear = 2013
#startYear = 2017
endYear = 2017

actual = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/iloilo_rice_data_cleaned_imputed_new.csv')
actual = actual[(actual.param == 'yield') & (actual.ecosystem == 'irri')].reset_index(drop=True)
actualFixed = actual.rename(columns={'muni':'ADM3_EN','year':'Date'}).drop(columns=['ecosystem'])
actualFixed= actualFixed[(actualFixed.Date <= endYear) & (actualFixed.Date >= startYear)].groupby(['ADM3_EN','param']).mean().reset_index(drop=False)
#actualFixed

points2 = points.loc[points['Class']=='Irrigated Rice'].groupby(['ADM3_EN']).mean().reset_index(drop=False)

merged = pd.merge(actualFixed, points2, left_on='ADM3_EN', right_on='ADM3_EN')
df = merged.rename(columns={'value':'actualYield','ch3_slope':'slope','ch3_elevat':'elevation'})
df = pd.DataFrame(df)

'''df = df.loc[df['ADM3_EN'].isin(['San Miguel',
'Oton',
'Barotac Viejo',
'Iloilo City',
'Leganes',
'Mina',
'Dingle',
'New Lucena',
'Santa Barbara',
'Pototan'])]
'''

'''df = df.loc[df['ADM3_EN'].isin([
'Zarraga',
'Anilao',
'Dumangas',
'Janiuay',
'Banate',
'Barotac Nuevo',
])]'''

#formula = 'actualYield ~ ndvi + LST_Night_1km + pcp + et + pet + slope + elevation'
formula = 'ndvi ~ lstday + pet + et'
results = smf.ols(formula = formula, 
                 missing='drop',
                 data=df).fit()
print(results.summary())
#print(pd.DataFrame(results.params).round(decimals=4).transpose())

model = forward_selected(df.drop(columns=['ADM3_EN','param','Date','fid_1','actualYield']), 'ndvi')
print('FORMULA: ', model.model.formula)
print('R2 = ', model.rsquared.round(decimals=4))

                            OLS Regression Results                            
Dep. Variable:                   ndvi   R-squared:                       0.502
Model:                            OLS   Adj. R-squared:                  0.395
Method:                 Least Squares   F-statistic:                     4.704
Date:                Tue, 05 Apr 2022   Prob (F-statistic):             0.0179
Time:                        01:24:11   Log-Likelihood:                 34.476
No. Observations:                  18   AIC:                            -60.95
Df Residuals:                      14   BIC:                            -57.39
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.8752      0.574      3.268      0.0

/Users/delatorredm/miniconda3/envs/ee/lib/python3.8/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [407]:
merged

,ADM3_EN,param,Date,value,fid_1,et,lstday,lstnight,pcp,pet,ch3_slope,ch3_elevat,ndvi
0,Anilao,yield,2015.0,4.107906,11101.000000,25.590000,28.662733,22.885086,408.795850,35.223333,0.679341,25.000000,1.604960
1,Banate,yield,2015.0,4.173094,10219.833333,21.171111,28.812414,23.148067,359.097152,30.767917,0.464795,9.000000,1.595563
2,Barotac Nuevo,yield,2015.0,4.259635,12258.409091,23.623589,29.783191,22.904002,406.627550,34.601364,0.429941,14.500000,1.610112
3,Barotac Viejo,yield,2015.0,4.255862,9512.571429,24.894722,29.598520,23.148506,377.845498,36.159841,0.973596,21.476190,1.549798
4,Dingle,yield,2015.0,4.596457,10817.111111,25.419120,29.633513,22.804544,420.999838,36.180694,0.656537,23.000000,1.582780
5,Dumangas,yield,2015.0,4.167609,13225.657895,27.093136,28.646812,22.665182,426.501846,38.766206,0.336499,7.947368,1.630101
6,Iloilo City,yield,2015.0,4.255819,14636.166667,22.801111,31.239275,23.378241,461.247458,35.088472,0.280022,9.833333,1.484503
7,Janiuay,yield,2015.0,4.126242,11681.000000,26.158333,29.478800,22.515848,412.824485,36.035000,0.000000,44.000000,1.575740
8,Leganes,yield,2015.0,4.518183,13976.923077,24.157051,29.532231,23.165378,449.547824,35.396667,0.235064,11.384615,1.588709
9,Mina,yield,2015.0,4.595758,11945.750000,27.930694,29.190509,22.967682,413.423742,38.094931,0.405078,32.666667,1.653747


In [379]:
X = df[[
    'pcp',
    #'lstday',
    'lstnight',
    'et',
    #'pet',
    #'slope',
    'elevation'
]]
y = df.ndvi
X = X.to_numpy()
y = y.to_numpy()

from sklearn.model_selection import train_test_split
random_state = 26
#11 #26 #6
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.20, 
                                                      random_state = random_state
                                                     )
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 50,
                           n_jobs = -1,
                           oob_score = True,
                           bootstrap = True,
                           random_state = random_state
                          )
rf.fit(X_train, y_train)

print('R^2 Training Score: {:.2f} \nOOB Score: {:.2f} \nR^2 Validation Score: {:.2f}'.format(rf.score(X_train, y_train), 
                                                                                             rf.oob_score_,
                                                                                             rf.score(X_valid, y_valid)))
print(rf.feature_importances_.round(decimals=4))

R^2 Training Score: 0.86 
OOB Score: -0.11 
R^2 Validation Score: 0.63
[0.53   0.1898 0.2052 0.0751]


In [400]:
y


array([1.56603143, 1.58377579, 1.57032353, 1.68639571, 1.52664667,
       1.60184118, 1.5066104 , 1.55561062, 1.52156471, 1.65604   ,
       1.568446  , 1.642614  , 1.50381529, 1.60972444, 1.651365  ,
       1.57446875, 1.626299  , 1.50114615, 1.54021   , 1.52190889,
       1.55625882, 1.60667111, 1.68122333, 1.51442   , 1.5553683 ,
       1.55685176, 1.65023818, 1.53577077, 1.64976133, 1.63408   ,
       1.44439867, 1.460944  , 1.63099176, 1.55533174, 1.6049475 ,
       1.5322728 , 1.49274   , 1.5560992 , 1.55421571, 1.56734843,
       1.52498636, 1.5622    , 1.56434   ])

## Rain-fed rice

In [408]:
points = '/Users/delatorredm/Library/CloudStorage/OneDrive-TheUniversityofAuckland/Ch4_Climate_Impacts/geospatial_layers/out5yr/intersect_DS_5yr.shp'
#points = '/Users/delatorredm/Library/CloudStorage/OneDrive-TheUniversityofAuckland/Ch4_Climate_Impacts/geospatial_layers/out2017/intersect_DS_2017.shp'

points = gpd.read_file(points)

startYear = 2013
endYear = 2017
actual = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/iloilo_rice_data_cleaned_imputed_new.csv')
actual = actual[(actual.param == 'yield') & (actual.ecosystem == 'rf')].reset_index(drop=True)
actualFixed = actual.rename(columns={'muni':'ADM3_EN','year':'Date'}).drop(columns=['ecosystem'])
actualFixed= actualFixed[(actualFixed.Date <= endYear) & (actualFixed.Date >= startYear)].groupby(['ADM3_EN','param']).mean().reset_index(drop=False)

points2 = points.loc[points['Class']=='Rain-fed Rice'].groupby(['ADM3_EN']).mean().reset_index(drop=False)

merged = pd.merge(actualFixed, points2, left_on='ADM3_EN', right_on='ADM3_EN')
merged

df = merged.rename(columns={'value':'actualYield','ch3_slope':'slope','ch3_elevat':'elevation'})
df = pd.DataFrame(df)

'''# RF I
df = df.loc[df['ADM3_EN'].isin([
'Alimodian',
'Cabatuan',
'Guimbal',
'Igbaras',
'Leganes',
'Leon',
'Maasin',
'Miagao',
'Oton',
'Pavia',
'San Joaquin',
'San Miguel',
'Santa Barbara',
'Tigbauan',
'Tubungan',
])]'''

'''# RF II
df = df.loc[df['ADM3_EN'].isin([
'Anilao',
'Badiangan',
'Banate',
'Barotac Nuevo',
'Barotac Viejo',
'Bingawan',
'Calinog',
'City of Passi',
'Dingle',
'Dumangas',
'Janiuay',
'Lambunao',
'Mina',
'New Lucena',
'Pototan',
'San Enrique',
'San Rafael',
'Zarraga',
])]'''

'''# RF III
df = df.loc[df['ADM3_EN'].isin([
'Ajuy',
'Balasan',
'Batad',
'Carles',
'Concepcion',
'Estancia',
'Lemery',
'San Dionisio',
])]
'''


#formula = 'actualYield ~ ndvi + LST_Night_1km + pcp + et + pet + slope + elevation'
formula = 'ndvi ~ pcp + et + slope + lstnight + 1'
results = smf.ols(formula = formula, 
                 missing='drop',
                 data=df).fit()
print(results.summary())
print(pd.DataFrame(results.params).round(decimals=4).transpose())

model = forward_selected(df.drop(columns=['ADM3_EN','param','Date','fid_1','actualYield']), 'ndvi')
print(model.model.formula)
print(model.rsquared)

                            OLS Regression Results                            
Dep. Variable:                   ndvi   R-squared:                       0.561
Model:                            OLS   Adj. R-squared:                  0.515
Method:                 Least Squares   F-statistic:                     12.15
Date:                Tue, 05 Apr 2022   Prob (F-statistic):           1.86e-06
Time:                        01:25:29   Log-Likelihood:                 80.168
No. Observations:                  43   AIC:                            -150.3
Df Residuals:                      38   BIC:                            -141.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.4926      0.233      6.393      0.0

In [409]:
merged

,ADM3_EN,param,Date,value,fid_1,et,lstday,lstnight,pcp,pet,ch3_slope,ch3_elevat,ndvi
0,Ajuy,yield,2015.0,3.532055,8097.828571,25.901476,28.201982,22.290354,436.523797,36.177619,2.308458,70.885714,1.566031
1,Alimodian,yield,2015.0,3.384596,13234.526316,26.777061,29.310358,22.330696,457.109342,37.350132,4.135539,161.947368,1.583776
2,Anilao,yield,2015.0,3.450418,10663.588235,24.302831,29.167668,22.730035,395.717009,34.684522,2.535541,46.941176,1.570324
3,Badiangan,yield,2015.0,3.881851,10635.642857,29.275387,28.981753,22.199447,458.468109,38.274405,1.676290,71.357143,1.686396
4,Balasan,yield,2015.0,3.087230,3157.555556,23.254244,28.501590,22.074684,561.967316,33.494190,2.001325,21.777778,1.526647
5,Banate,yield,2015.0,3.841311,9821.941176,25.004779,29.448144,22.796098,371.757514,36.216127,1.544544,34.352941,1.601841
6,Barotac Nuevo,yield,2015.0,3.327693,12045.880000,20.876342,26.287933,20.038609,427.727207,30.314958,0.334543,7.960000,1.506610
7,Barotac Viejo,yield,2015.0,3.459721,9006.750000,26.309505,29.415561,22.614315,411.930386,36.631992,2.789280,67.906250,1.555611
8,Batad,yield,2015.0,3.580798,3869.823529,24.839412,29.387400,22.730414,555.838455,36.366765,3.641028,39.588235,1.521565
9,Bingawan,yield,2015.0,3.307013,7262.000000,29.173611,29.319221,21.811331,519.972665,39.792222,1.189152,87.166667,1.656040


In [394]:
X = df[[
    'pcp',
    'lstday',
    #'lstnight',
    'et',
    #'pet',
    'slope',
    #'elevation'
]]
y = df.ndvi
X = X.to_numpy()
y = y.to_numpy()

from sklearn.model_selection import train_test_split
random_state = 26
#11 #26 #6
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.20, 
                                                      random_state = random_state
                                                     )
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 50,
                           n_jobs = -1,
                           oob_score = True,
                           bootstrap = True,
                           random_state = random_state
                          )
rf.fit(X_train, y_train)

print('R^2 Training Score: {:.2f} \nOOB Score: {:.2f} \nR^2 Validation Score: {:.2f}'.format(rf.score(X_train, y_train), 
                                                                                             rf.oob_score_,
                                                                                             rf.score(X_valid, y_valid)))
print(rf.feature_importances_.round(decimals=4))

R^2 Training Score: 0.95 
OOB Score: 0.52 
R^2 Validation Score: 0.70
[0.1718 0.0813 0.6839 0.063 ]


## ALL RICE

In [44]:
points = '/Users/delatorredm/Library/CloudStorage/OneDrive-TheUniversityofAuckland/Ch4_Climate_Impacts/geospatial_layers/out5yr/intersect_DS_5yr.shp'
#points = '/Users/delatorredm/Library/CloudStorage/OneDrive-TheUniversityofAuckland/Ch4_Climate_Impacts/geospatial_layers/out2017/intersect_DS_2017.shp'

points = gpd.read_file(points)

startYear = 2013
endYear = 2017
actual = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/iloilo_rice_data_cleaned_imputed_new.csv')

irrlist = [
'Anilao',
'Banate',
'Barotac Nuevo',
'Barotac Viejo',
'Dingle',
'Dumangas',
'Iloilo City',
'Janiuay',
'Leganes',
'Mina',
'New Lucena',
'Oton',
'Pavia',
'Pototan',
'San Miguel',
'Santa Barbara',
'Tigbauan',
'Zarraga',
]

list1 = actual[(actual.param == 'yield') & (actual.ecosystem == 'rf')].reset_index(drop=True)
list1 = list1.loc[~list1['muni'].isin(irrlist)]

list2 = actual[(actual.param == 'yield') & (actual.ecosystem == 'all')].reset_index(drop=True)
list2 = list2.loc[list2['muni'].isin(irrlist)]
actual = pd.concat([list1, list2])

actualFixed = actual.rename(columns={'muni':'ADM3_EN','year':'Date'}).drop(columns=['ecosystem']).reset_index(drop=True)
actualFixed= actualFixed[(actualFixed.Date <= endYear) & (actualFixed.Date >= startYear)].groupby(['ADM3_EN','param']).mean().reset_index(drop=False)
actualFixed

points2 = points.groupby(['ADM3_EN']).mean().reset_index(drop=False)

merged = pd.merge(actualFixed, points2, left_on='ADM3_EN', right_on='ADM3_EN')
df = merged.rename(columns={'value':'actualYield','ch3_slope':'slope','ch3_elevat':'elevation'})
df = pd.DataFrame(df)


#formula = 'actualYield ~ ndvi + LST_Night_1km + pcp + et + pet + slope + elevation'
formula = 'ndvi ~ et + pet + lstnight + pcp + 1'
results = smf.ols(formula = formula, 
                 missing='drop',
                 data=df).fit()
print(results.summary())
print(pd.DataFrame(results.params).round(decimals=4).transpose())

model = forward_selected(df.drop(columns=['ADM3_EN','param','Date','fid_1','actualYield']), 'ndvi')
print(model.model.formula)
print(model.rsquared)

                            OLS Regression Results                            
Dep. Variable:                   ndvi   R-squared:                       0.761
Model:                            OLS   Adj. R-squared:                  0.737
Method:                 Least Squares   F-statistic:                     31.08
Date:                Sat, 12 Mar 2022   Prob (F-statistic):           1.18e-11
Time:                        16:32:44   Log-Likelihood:                 101.13
No. Observations:                  44   AIC:                            -192.3
Df Residuals:                      39   BIC:                            -183.3
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.1565      0.185      6.244      0.0

In [47]:
X = df[['pcp','lstday','lstnight','et','pet','slope','elevation']]
y = df.ndvi
X = X.to_numpy()
y = y.to_numpy()

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state = 1)
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 1000,
                           #n_jobs = -1,
                           oob_score = True,
                           #bootstrap = True,
                           random_state = 1)
rf.fit(X_train, y_train)

print('R^2 Training Score: {:.2f} \nOOB Score: {:.2f} \nR^2 Validation Score: {:.2f}'.format(rf.score(X_train, y_train), 
                                                                                             rf.oob_score_,
                                                                                             rf.score(X_valid, y_valid)))
print(rf.feature_importances_.round(decimals=4))

R^2 Training Score: 0.92 
OOB Score: 0.34 
R^2 Validation Score: 0.56
[0.17   0.0749 0.047  0.4947 0.0936 0.0555 0.0643]
